In [95]:
import pandas as pd
import csv
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom

In [ ]:
df = pd.read_excel('./data.xlsx')
df.head()

In [48]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]

In [81]:
basket = (df[df['Country'] =="United Kingdom"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [82]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket.drop('POSTAGE', inplace=True, axis=1)

In [83]:
df['Description'].shape

(532621,)

In [84]:
# basket[0:1].shape
# print(basket[0:1]["ACRYLIC JEWEL ANGEL,PINK"])
# print()
# for i in basket[0:1][:40]:
#     print(i)

In [88]:
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

In [100]:
# support  min threshold(0.01)，confidence  min threshold(0.5)
rules1 = association_rules(frequent_itemsets, metric="support", min_threshold= 0.01)
rules1.head()

antecedants               consequents  antecedent support  \
0            (DOTCOM POSTAGE)  (6 RIBBONS RUSTIC CHARM)            0.037928   
1    (6 RIBBONS RUSTIC CHARM)          (DOTCOM POSTAGE)            0.045803   
2    (JAM MAKING SET PRINTED)  (6 RIBBONS RUSTIC CHARM)            0.053999   
3    (6 RIBBONS RUSTIC CHARM)  (JAM MAKING SET PRINTED)            0.045803   
4  (JAM MAKING SET WITH JARS)  (6 RIBBONS RUSTIC CHARM)            0.053517   

   consequent support   support  confidence      lift  leverage  conviction  
0            0.045803  0.010821    0.285311  6.229118  0.009084    1.335122  
1            0.037928  0.010821    0.236257  6.229118  0.009084    1.259681  
2            0.045803  0.011571    0.214286  4.678446  0.009098    1.214433  
3            0.053999  0.011571    0.252632  4.678446  0.009098    1.265776  
4            0.045803  0.010339    0.193193  4.217938  0.007888    1.182684

In [101]:
rules2 = association_rules(frequent_itemsets, metric="confidence", min_threshold= 0.5)
rules2.head()

antecedants                           consequents  \
0  (60 CAKE CASES DOLLY GIRL DESIGN)  (PACK OF 60 PINK PAISLEY CAKE CASES)   
1  (60 CAKE CASES DOLLY GIRL DESIGN)     (PACK OF 72 RETROSPOT CAKE CASES)   
2      (60 TEATIME FAIRY CAKE CASES)     (PACK OF 72 RETROSPOT CAKE CASES)   
3   (ALARM CLOCK BAKELIKE CHOCOLATE)          (ALARM CLOCK BAKELIKE GREEN)   
4   (ALARM CLOCK BAKELIKE CHOCOLATE)            (ALARM CLOCK BAKELIKE RED)   

   antecedent support  consequent support   support  confidence       lift  \
0            0.019875            0.041839  0.010125    0.509434  12.176189   
1            0.019875            0.060481  0.010928    0.549865   9.091527   
2            0.040339            0.060481  0.021910    0.543161   8.980674   
3            0.020357            0.046928  0.013339    0.655263  13.963239   
4            0.020357            0.049821  0.013607    0.668421  13.416576   

   leverage  conviction  
0  0.009293    1.953175  
1  0.009726    2.087195  
2  0.019471    2.056563  
3  0.012384    2.764637  
4  0.012593    2.865621

In [105]:
result1 = rules2[ (rules2['support'] >= 0.01) &
       (rules2['confidence'] >= 0.5) ]
print(result1.shape)
result1.head()

(1055, 9)


antecedants                           consequents  \
0  (60 CAKE CASES DOLLY GIRL DESIGN)  (PACK OF 60 PINK PAISLEY CAKE CASES)   
1  (60 CAKE CASES DOLLY GIRL DESIGN)     (PACK OF 72 RETROSPOT CAKE CASES)   
2      (60 TEATIME FAIRY CAKE CASES)     (PACK OF 72 RETROSPOT CAKE CASES)   
3   (ALARM CLOCK BAKELIKE CHOCOLATE)          (ALARM CLOCK BAKELIKE GREEN)   
4   (ALARM CLOCK BAKELIKE CHOCOLATE)            (ALARM CLOCK BAKELIKE RED)   

   antecedent support  consequent support   support  confidence       lift  \
0            0.019875            0.041839  0.010125    0.509434  12.176189   
1            0.019875            0.060481  0.010928    0.549865   9.091527   
2            0.040339            0.060481  0.021910    0.543161   8.980674   
3            0.020357            0.046928  0.013339    0.655263  13.963239   
4            0.020357            0.049821  0.013607    0.668421  13.416576   

   leverage  conviction  
0  0.009293    1.953175  
1  0.009726    2.087195  
2  0.019471    2.056563  
3  0.012384    2.764637  
4  0.012593    2.865621

In [177]:
result2 = result1

# test_res_1 = result2[:1]["antecedants"].values[0]
# slipt_str = test_res_1.split("{'")[1].split("'}")[0]

# teststr = "60 CAKE CASES DOLLY GIRL DESIGN"

# print( slipt_str, teststr, "\n",  slipt_str == teststr, "\n")
# print(result1["consequents"][:1])

In [200]:
# create the result csv
import numpy as np
submit_data = pd.read_csv('submit.csv')
print(submit_data.shape)
submit_data.head()

print(submit_data["Association Rule antecedants"][:3][0])
print(submit_data["Association Rule antecedants"][:3])


index = 1
final_result = np.zeros(3055)
print(type(result2), result2[:3], result2.shape)


for ant_res2,con_res2 in zip(result2["antecedants"], result2["consequents"]):
#     for sub in submit_data:
        print("\n\n res2", type(ant_res2), ant_res2)
        print("\n\n res2", type(con_res2), con_res2 )
#         test_res_1 = res2["antecedants"].values[0]
#         slipt_str1 = test_res_1.split("{'")[1].split("'}")[0]
        
#         test_res_2 = res2["consequents"].values[0]
#         slipt_str2 = test_res_1.split("{'")[1].split("'}")[0]
        
#         for sub1 in sub["Association Rule antecedants"]:
#             if sub1 == slipt_str1:
#                 for sub2 in sub["Association Rule consequents"]:
#                     if sub2 == slipt_str2: 
#                         final_result[index] = 1
#                         print("1")
        index += 1
        

(3055, 4)
WOODLAND CHARLOTTE BAG, CHARLOTTE BAG PINK POLKADOT
0    WOODLAND CHARLOTTE BAG, CHARLOTTE BAG PINK POL...
1                            FRENCH ENAMEL WATER BASIN
2       NATURAL SLATE HEART CHALKBOARD, DOTCOM POSTAGE
Name: Association Rule antecedants, dtype: object
<class 'pandas.core.frame.DataFrame'>                                       antecedants  \
0  frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})   
1  frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})   
2  frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})   

                            consequents  antecedent support  \
0  (PACK OF 60 PINK PAISLEY CAKE CASES)            0.019875   
1     (PACK OF 72 RETROSPOT CAKE CASES)            0.019875   
2     (PACK OF 72 RETROSPOT CAKE CASES)            0.040339   

   consequent support   support  confidence       lift  leverage  conviction  
0            0.041839  0.010125    0.509434  12.176189  0.009293    1.953175  
1            0.060481  0.010928    0.549865   9.091527  0.009



 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({'CHARLOTTE BAG SUKI DESIGN'})


 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({'WOODLAND CHARLOTTE BAG'})


 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({'CHARLOTTE BAG SUKI DESIGN'})


 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({'RED RETROSPOT CHARLOTTE BAG'})


 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({'CHARLOTTE BAG SUKI DESIGN'})


 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({'LUNCH BAG RED RETROSPOT'})


 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({'RED RETROSPOT CHARLOTTE BAG'})


 res2 <class 'str'> frozenset({'6

 res2 <class 'frozenset'> frozenset({'JUMBO BAG PINK POLKADOT'})


 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({'JUMBO BAG RED RETROSPOT'})


 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({'JUMBO BAG PINK POLKADOT'})


 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({'JUMBO BAG RED RETROSPOT'})


 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({'JUMBO BAG PINK POLKADOT'})


 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({'JUMBO BAG RED RETROSPOT'})


 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({'JUMBO BAG PINK POLKADOT'})


 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({'JUMBO BAG 



 res2 <class 'frozenset'> frozenset({"POPPY'S PLAYHOUSE KITCHEN", "POPPY'S PLAYHOUSE LIVINGROOM"})


 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({"POPPY'S PLAYHOUSE KITCHEN", "POPPY'S PLAYHOUSE BEDROOM"})


 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({'STRAWBERRY CHARLOTTE BAG'})


 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({'RED RETROSPOT CHARLOTTE BAG'})


 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({'WOODLAND CHARLOTTE BAG'})


 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({'REGENCY TEA PLATE ROSES'})


 res2 <class 'str'> frozenset({'60 CAKE CASES DOLLY GIRL DESIGN'})


 res2 <class 'frozenset'> frozenset({'REGENCY TEA PLATE GREEN'})


 res2 <class 'str'> frozenset({'60 CAKE CASES 

In [20]:
basket2 = (df[df['Country'] =="Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.05, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)

rules2[ (rules2['lift'] >= 4) &
        (rules2['confidence'] >= 0.5)]

Empty DataFrame
Columns: [antecedants, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction]
Index: []

Empty DataFrame
Columns: [antecedants, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction]
Index: []